In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression
import torch
import torch.optim as optim
import torch.nn as nn
import datetime
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

In [5]:
class StepByStep(object):
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.to(self.device)
        self.losses = []
        self.val_loss = []
        self.epochs = 0
        self.train_step_fn = self.train_step_fn()
        self.val_step_fn = self.val_step_fn()
    def to(self, device):
        try:
            self.device = device

            self.model.to(self.device)

        except RuntimeError:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            print("can't send it to ${device}, sending to ${self.device} instead")
            self.model.to(self.device)
            self.train_loader = None
            self.valid_loader = None
            self.writer = None
    def set_loaders(self, train_loader, val_loader = None):
        self.train_loader = train_loader
        self.valid_loader = val_loader
    def set_tensorboard(self, name, folder='runs'):
        suffix = datetime.datetime.now().strftime('%Y%d%H%M%S')
        self.writer = SummaryWriter(f'{folder}/{name}_{suffix}')